In [1]:
import os
import random
import torch.nn

from training.inference import get_latents

def generate_evaluation_dataset(model, dataloader, name, chunking=True, averaging=False, chunk_size=256):
    model.mask_ratio = 0.0
    latents, labels = get_latents(dataloader, model, chunking=chunking, averaging=averaging, chunk_size=chunk_size)
    print("Saving...")

    index = 0
    r = random.Random()

    counts = [0] * 10
    for latent, label in zip(latents, labels):
        directory = f"D:\\SongsDataset\\GTZAN\\latent_datasets\\{name}\\"

        if r.random() < 0.1 and counts[label] < 10:
            directory += "test\\"
            counts[0] += 1
        else:
            directory += "train\\"

        os.makedirs(os.path.dirname(directory), exist_ok=True)

        if not averaging and chunking:
            for x in range(label.shape[0]):
                label_name = directory + f"{index}_label"
                latent_name = directory + f"{index}_data"
                torch.save(latent, latent_name + ".pt")
                torch.save(label, label_name + ".pt")
                index += 1
        else:
            label_name = directory + f"{index}_label"
            latent_name = directory + f"{index}_data"
            torch.save(latent, latent_name + ".pt")
            torch.save(label, label_name + ".pt")
            index += 1

In [ ]:
from data.data_utils import GTZAN
from torch.utils.data import DataLoader
import torch.nn

directory = "D:\\SongsDataset\\GTZAN\\genres_original\\"
GTZAN_dataset = GTZAN(directory)
GTZAN_dataloader = DataLoader(
    GTZAN_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=2,
    prefetch_factor=1
)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [19]:
model = torch.load("E:\\Coding\\SongAnalyzer\\Analyzer\\src\\trained_models\\Myna-CLS-Sinusoidal\\Epoch-79.pt", weights_only=False)
generate_evaluation_dataset(model, GTZAN_dataloader, "Sinusoidal-Chunking-256", chunking=True, chunk_size=256, averaging=True)

100%|██████████| 999/999 [02:28<00:00,  6.73it/s]  


Saving...


In [20]:
model = torch.load("E:\\Coding\\SongAnalyzer\\Analyzer\\src\\trained_models\\Myna-CLS-ALIBI\\Epoch-103.pt", weights_only=False)
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-256", chunking=True, chunk_size=256, averaging=True)

  0%|          | 0/999 [01:05<?, ?it/s]


AttributeError: 'Alibi2DBias' object has no attribute 'only_x'

In [27]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-512", chunking=True, chunk_size=512, averaging=True)

100%|██████████| 999/999 [00:35<00:00, 28.14it/s]


Saving...


In [28]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-1024", chunking=True, chunk_size=1024, averaging=True)

100%|██████████| 999/999 [00:29<00:00, 34.35it/s]


Saving...


In [29]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-2048", chunking=True, chunk_size=2048, averaging=True)

100%|██████████| 999/999 [00:38<00:00, 26.22it/s]


Saving...


In [17]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-4096", chunking=True, chunk_size=4096)

  0%|          | 0/999 [00:15<?, ?it/s]


RuntimeError: max(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument.

In [15]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-No-Chunking", chunking=False, chunk_size=16384)

  0%|          | 0/999 [00:16<?, ?it/s]


RuntimeError: max(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument.

In [9]:
from data.data_utils import LatentDataset
import torch
from datasets import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# ----- Training Loop -----
def train_model(model, train_dataloader, test_dataloader, config, device="cuda"):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    model.train()
    for epoch in tqdm(range(config.num_epochs)):
        total_loss = 0
        for labels, data in train_dataloader:
            # if dataset returns one-hot, convert back to integer for CrossEntropy
            if labels.ndim > 1:
                labels = labels.argmax(dim=1)

            data = data.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        if epoch % 64 == 0:
            print(f"Epoch {epoch+1}/{config.num_epochs}, Loss: {total_loss/len(train_dataloader):.4f}")
            evaluate_model(model, test_dataloader, config, device)
            model.train()

    return model

# Evaluation
def evaluate_model(model, dataloader, config, device="cuda"):
    model.eval()
    all_predictions = []
    all_labels = []

    for labels, data in tqdm(dataloader):
        # if dataset returns one-hot, convert back to integer for CrossEntropy
        if labels.ndim > 1:
            labels = labels.argmax(dim=1)

        data = data.to(device)
        labels = labels.to(device)

        outputs = model(data)
        predicted = outputs.argmax(dim=1)

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    scores = f1_score(all_labels, all_predictions, average="macro")
    accuracy = accuracy_score(all_labels, all_predictions)

    print(f"F1: {scores}\nAccuracy: {accuracy:.4f}")


In [10]:
import torch.nn
from torch import nn, optim

class MLP(nn.Module):
    def __init__(self, input=128, output=10, dropout=0.1):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input, 512),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(512, output)
        )

    def forward(self, x):
        return self.model(x)

class LinearProbe(nn.Module):
    def __init__(self, input=128, output=10):
        super().__init__()
        self.model = nn.Linear(input, output)

    def forward(self, x):
        return self.model(x)

In [11]:
from training.evaluation import local_coherence
from utils.Config import Config
from torch.utils.data import DataLoader
import numpy as np

def test_latent_dataaset(dataset):
    directory = f"D:\\SongsDataset\\GTZAN\\latent_datasets\\{dataset}\\"

    num_classes = 10
    train_latent_dataset = LatentDataset(directory + "train\\", num_classes=num_classes)
    train_latent_dataloader = DataLoader(
        train_latent_dataset,
        batch_size=64,
        shuffle=True,
    )

    test_latent_dataset = LatentDataset(directory + "test\\", num_classes=num_classes)
    test_latent_dataloader = DataLoader(
        test_latent_dataset,
        batch_size=64,
        shuffle=True,
    )

    lgc = local_coherence(np.array(train_latent_dataloader.latents), np.array(train_latent_dataloader.labels), k=50)
    print(f"Local Coherence: {lgc}")

    model_name = f"LinearClassifier-{dataset}"
    config = Config(
            save_path=f"trained_models\\{model_name}\\",
            num_epochs=512,
            learning_rate=1e-3,
            weight_decay=1e-3,
            num_workers=2,
            batch_size= 64,
            eval_batch_size=64,
            dtype=torch.float32
        )

    device = "cuda"
    model = MLP(128, num_classes).to(device)
    train_model(model, train_latent_dataloader, config, device=device)
    evaluate_model(model, test_latent_dataloader, config, device=device)

In [12]:
dataset = "Sinusoidal-Chunking-256"
test_latent_dataaset(dataset)

Local Coherence: 0.4187387387387388


  0%|          | 0/512 [00:00<?, ?it/s]

Epoch 1/512, Loss: 1.7567
Epoch 65/512, Loss: 0.0227
Epoch 129/512, Loss: 0.0099
Epoch 193/512, Loss: 0.0043
Epoch 257/512, Loss: 0.0052
Epoch 321/512, Loss: 0.0034
Epoch 385/512, Loss: 0.0013
Epoch 449/512, Loss: 0.0030


  0%|          | 0/16 [00:00<?, ?it/s]

F1: 0.998999974999375
Accuracy: 0.9990


In [13]:
dataset = "ALIBI-Chunking-256"
test_latent_dataaset(dataset)

Local Coherence: 0.4216416416416417


  0%|          | 0/512 [00:00<?, ?it/s]

Epoch 1/512, Loss: 1.7324
Epoch 65/512, Loss: 0.0262
Epoch 129/512, Loss: 0.0080
Epoch 193/512, Loss: 0.0049
Epoch 257/512, Loss: 0.0070
Epoch 321/512, Loss: 0.0064
Epoch 385/512, Loss: 0.0030
Epoch 449/512, Loss: 0.0036


  0%|          | 0/16 [00:00<?, ?it/s]

F1: 0.998999974999375
Accuracy: 0.9990


In [14]:
dataset = "ALIBI-Chunking-512"
test_latent_dataaset(dataset)

Local Coherence: 0.4152552552552553


  0%|          | 0/512 [00:00<?, ?it/s]

Epoch 1/512, Loss: 1.6886
Epoch 65/512, Loss: 0.0343
Epoch 129/512, Loss: 0.0085
Epoch 193/512, Loss: 0.0085
Epoch 257/512, Loss: 0.0036
Epoch 321/512, Loss: 0.0419
Epoch 385/512, Loss: 0.0014
Epoch 449/512, Loss: 0.0380


  0%|          | 0/16 [00:00<?, ?it/s]

F1: 0.998999974999375
Accuracy: 0.9990


In [15]:
dataset = "ALIBI-Chunking-1024"
test_latent_dataaset(dataset)

Local Coherence: 0.4057457457457458


  0%|          | 0/512 [00:00<?, ?it/s]

Epoch 1/512, Loss: 1.7307
Epoch 65/512, Loss: 0.0362
Epoch 129/512, Loss: 0.0082
Epoch 193/512, Loss: 0.0043
Epoch 257/512, Loss: 0.0042
Epoch 321/512, Loss: 0.0019
Epoch 385/512, Loss: 0.0035
Epoch 449/512, Loss: 0.0029


  0%|          | 0/16 [00:00<?, ?it/s]

F1: 0.998999974999375
Accuracy: 0.9990


In [16]:
dataset = "ALIBI-Chunking-2048"
test_latent_dataaset(dataset)

Local Coherence: 0.4044644644644645


  0%|          | 0/512 [00:00<?, ?it/s]

Epoch 1/512, Loss: 1.7127
Epoch 65/512, Loss: 0.0315
Epoch 129/512, Loss: 0.0095
Epoch 193/512, Loss: 0.0052
Epoch 257/512, Loss: 0.0076
Epoch 321/512, Loss: 0.0021
Epoch 385/512, Loss: 0.0019
Epoch 449/512, Loss: 0.0093


  0%|          | 0/16 [00:00<?, ?it/s]

F1: 0.998999974999375
Accuracy: 0.9990
